# Макросы в C++

С помощью директивы #define в C++ можно создавать макросы.

Объявление макросы выглядит так:

In [1]:
#define macros expression

После создания макроса компилятор при работе будет заменять все выражения macros на expression.

In [2]:
#include <iostream>
using namespace std;

In [3]:
#define hello_world "Hello, world!"

In [4]:
cout << hello_world << endl; // преобразуется в cout << "Hello, world!" << endl;

Hello, world!


In [5]:
#define INF_INT (1 << 30) // олимпиадный макрос

In [6]:
cout << INF_INT << endl;

1073741824


In [7]:
cout << (1 < INF_INT) << " " << (1203 < INF_INT) << endl;

1 1


Также можно создавать "макросы-функции".

In [8]:
#define MAX(a, b) a > b ? a : b
#define MIN(a, b) a < b ? a : b

In [9]:
cout << (MAX(1, 2)) << " " << (MIN(5, 10)) << endl;

2 5


Можно делать более сложные макросы.

In [10]:
#define KeyValuePrint(key, value) key << " = " << value

In [11]:
int x = 10;
cout << KeyValuePrint("x", x);

x = 10

In [12]:
cout << KeyValuePrint("x", x) << '!' << endl;

x = 10!


In [13]:
cout << "Hello, " << KeyValuePrint("x", x) << '!' << endl;

Hello, x = 10!


Многострочные макросы.

In [14]:
#define SUM_AND_LOG(a, b) \
cout << #a << " = " << (a) << ", ";\
cout << #b << " = " << (b) << ", ";\
cout << #a << " + " << #b << " = " << (a) + (b);

In [15]:
int x = 10, y = 20;
SUM_AND_LOG(x, y)

x = 10, y = 20, x + y = 30

@0x7f37af553ba0

In [16]:
#define LOG_OPERATION(a, b, OPERATOR) \
cout << #a << " = " << (a) << ", ";\
cout << #b << " = " << (b) << ", ";\
cout << #a << ' ' << #OPERATOR << ' ' << #b << " = " << (a) OPERATOR (b);

In [17]:
LOG_OPERATION(10, 20, +)

10 = 10, 20 = 20, 10 + 20 = 30

@0x7f37af553ba0

In [18]:
LOG_OPERATION(10, 20, -)

10 = 10, 20 = 20, 10 - 20 = -10

@0x7f37af553ba0

## Оператор \#

Данный оператор объект как текст.

In [19]:
#define KVPrint(x) #x << " = " << x

In [20]:
cout << KVPrint(x) << endl;

x = 10


In [21]:
int y = 20, z = 30;

In [22]:
cout << KVPrint(x) << ", " << KVPrint(y) << ", " << KVPrint(z) << endl;

x = 10, y = 20, z = 30


## Макрос \_\_FILE__

Данный макрос возвращает путь от корневой директории до текущего файла.

In [23]:
const char *this_file = __FILE__;

In [24]:
cout << this_file << endl;
// смотреть в нормальном C++

input_line_35


@0x7f37af553ba0

## Макрос \_\_LINE__

Данный макрос возвращает номер строки в файле, в которой он был вызван.

In [25]:
int line = __LINE__;

In [26]:
cout << line << endl;
// смотреть в нормальном C++

2


@0x7f37af553ba0

## Почему макросы - зло?

In [27]:
#define MAX(a, b) a > b ? a : b

In [28]:
int a = 10, b = 20;

In [29]:
int max_num = MAX(a, b);

In [30]:
cout << max_num;

20

Всё работает правильно, но что будет, если передать b++?

In [31]:
int max_num = MAX(a, b++);

In [32]:
cout << max_num;

21

И снова всё работает верно, но посмотрим на переменные a и b:

In [33]:
printf("%d %d", a, b);

10 22

b == 22! Но b++ должен был дать 21. Почему 22?

На самом деле макрос MAX(a, b) преобразуется в выражение a > b ? a : b, 
причём a и b не переменные, а какой-либо текст.

Соответственно, выражение MAX(a, b++) преобразуется не в a > b ? a : b.
Оно преобразуется в a > b++ ? a : b++.
И тогда b++ вызывается 2 раза.
Но т. к. используется постинкремент, то данный оператор увеличивает значение b на 1, а возвращает предыдущее значение b, поэтому результат выполнения макроса оказался верным, но его выполнение может привести к труднонаходимым ошибкам.

**Как исправить?** -Использовать функцию.

In [34]:
template<typename T>
T max(T a, T b) { return a > b ? a : b; }

Данная функция будет работать корректно, т. к. сначала будет вычислены аргументы a и b, 
а только после этого начнётся выполнение функции.

Также в "макросах-функциях" с аргументами желательно использовать скобки в выражении при действиях с аргументами.

In [35]:
#define LOG(expression) cout << expression << endl;

In [36]:
LOG(10); //  OK

10


@0x7f37af553ba0

In [37]:
LOG(123.456); // OK

123.456


@0x7f37af553ba0

In [38]:
LOG(1 << 4); // ? 1 << 4 == 16

14


@0x7f37af553ba0

Компилятор подставил "1 << 4" в выражение "cout << expression << endl;".
Итог: "cout << 1 << 4 << endl;".

Здесь оператор битового сдвига влево был распознан компилятором, 
как оператор << объекта ostream для двух целых чисел,
поэтому результатом является вывод 1 и 4, а не числа 14.

Чтобы решить данную проблему, нужно заключить expression в скобки:

In [39]:
#define LOG_FIXED(expression) cout << (expression) << endl;

In [40]:
LOG_FIXED(10); // OK

10


@0x7f37af553ba0

In [41]:
LOG_FIXED(123.456); // OK

123.456


@0x7f37af553ba0

In [42]:
LOG_FIXED(1 << 4); // OK

16


@0x7f37af553ba0

## Итог

Если есть возможность не использовать макрос, то ей надо воспользоваться.